# <h1><center>Smart Attendance System using OpenCV - Face recognition and attendance marking</center></h1>
<center>By - Swayansu Mishra  </center>

As we discussed in the previous notebook the system consists of several key steps:

1. Capturing images of individuals for training purposes
2. Preprocessing the images and creating labels
3. Splitting the dataset into training and testing sets
4. Defining, compiling, and training a Convolutional Neural Network (CNN) model
5. Marking attendance for recognized individuals in a CSV file
6. Implementing a real-time face recognition system using the trained model
7. Throughout this notebook, we provide detailed explanations, comments, and code sections to guide you through the development process.

Out of all these we covered step 1 to 4 already. we will now focus on the remaining steps.

## Import Libraries

First, we need to import the necessary libraries for the project:

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from datetime import datetime
import time
import pickle
from keras.models import load_model

## Mark Attendance

Next, we will create a function to mark attendance in a CSV file for the recognized person.

### Function to mark attendance in a CSV file

This function marks attendance for a recognized person by adding a new record with their name and the current time to the CSV file. If the attendance file for the current date does not exist, it creates a new file with appropriate headers.

In [ ]:
def mark_attendance(name):
    current_date = datetime.now().strftime('%Y-%m-%d')
    filename = f"attendance_{current_date}.csv"
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # Create a new CSV file if it does not exist
    if not os.path.exists(filename):
        with open(filename, 'w') as f:
            f.write('Name,Time\n')

    # Read the CSV file and check if the person's attendance is already marked
    df = pd.read_csv(filename)

    # If the person's attendance is not marked, add a new record with the person's name and current time
    if name not in df['Name'].values:
        new_record = {'Name': name, 'Time': current_time}
        df = df.append(new_record, ignore_index=True)
        df.to_csv(filename, index=False)
        print(f"Attendance marked for {name}")

## Real-time Attendance System

Finally, we will create a function to start the real-time attendance system using the trained CNN model. This function will continuously capture video frames from the webcam, recognize the person's face, and mark attendance accordingly.

We load the pre-trained Haar Cascade classifier for detecting frontal faces, which is available in the OpenCV library as `haarcascade_frontalface_default.xml`.

Haar Cascade classifiers are based on the Haar-like features proposed by Paul Viola and Michael Jones in their paper, "Rapid Object Detection using a Boosted Cascade of Simple Features." Haar-like features are simple rectangular patterns that can be used to detect edges, lines, and other patterns in images.

A Haar Cascade classifier is a collection of multiple stages, each containing a set of weak classifiers. These weak classifiers are simple classifiers that can detect specific patterns with low accuracy. In each stage, the weak classifiers are combined into a stronger classifier using the AdaBoost algorithm. The classifiers in later stages are more complex and better at detecting the target object than those in earlier stages. During the detection process, an image is passed through these stages in a cascade, and if it passes all the stages, the object is detected.

The `haarcascade_frontalface_default.xml` file is a pre-trained Haar Cascade classifier provided by OpenCV to detect frontal faces in images. This classifier has been trained on a large dataset of face and non-face images, and it can effectively detect faces in real-time when used with the OpenCV library.

### Function to start the real-time attendance system

In this function, we capture video frames from the webcam and process each frame to detect faces, recognize the person, and mark attendance. The detected faces are highlighted with a bounding box, and the recognized person's name is displayed above the box.

In [ ]:
def start_attendance(model_path, encoder):
    # Load the saved model
    model = load_model(model_path)
    # Add cooldown period and cooldown_timestamps here
    cooldown_period = 10
    cooldown_timestamps = {}
    
    cap = cv2.VideoCapture('http://172.16.1.169:4747/video')
#     cap = cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Loop to process the video frames
    while True:
        # Read frames from the webcam
        ret, frame = cap.read()
        # Convert the frames to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Detect faces in the grayscale frames
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

        # Loop through the detected faces
        for (x, y, w, h) in faces:
            # Extract the region of interest (face) from the grayscale frame
            roi_gray = gray_frame[y:y + h, x:x + w]
            # Resize the face to match the input size of the CNN
            resized_roi = cv2.resize(roi_gray, (96, 96))
            # Convert the resized_roi to 3 channels
            resized_roi_3_channels = np.repeat(resized_roi[..., np.newaxis], 3, axis=-1)
            # Normalize the face and convert it to the required format for the CNN
            roi_input = resized_roi_3_channels.reshape(1, 96, 96, 3).astype('float32') / 255.0

            # Use the trained model to predict the person's name
            confidence_threshold = 0.6
            predictions = model.predict(roi_input)
            label = np.argmax(predictions)
            confidence = predictions[0][label]

            if confidence > confidence_threshold:
                name = encoder.inverse_transform([label])[0]
                current_time = time.time()
                
                if name not in cooldown_timestamps or (current_time - cooldown_timestamps[name] >= cooldown_period):
                    mark_attendance(name)
                    cooldown_timestamps[name] = current_time
            else:
                name = "Unknown"
            
            # Draw a rectangle around the detected face and display the person's name
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(frame, name, (x + 6, y - 6), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255), 1)

        # Display the processed frames
        cv2.imshow('Attendance System', frame)

        # Exit the loop if the user presses 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and close the windows
    cap.release()
    cv2.destroyAllWindows()

# Start the attendance system with the saved model
start_attendance('face_recognition_model.h5', encoder)

NameError: name 'encoder' is not defined

We will now load the saved encoder and start the attendance system with the saved model.

In [ ]:
# Load the saved encoder
with open('label_encoder.pkl', 'rb') as f:
    loaded_encoder = pickle.load(f)
    
# Start the attendance system with the saved model
start_attendance('face_recognition_model.h5', loaded_encoder)

## Conclusion

In this project, we developed a Smart Attendance System that uses OpenCV and a Convolutional Neural Network to recognize faces and mark attendance in real-time. By capturing images of multiple people, training a model to recognize their faces, and implementing a real-time attendance system using a webcam, we have created an efficient and automated way to track attendance.

We have also briefly discussed the Haar Cascade classifier and the `haarcascade_frontalface_default.xml` file, which is a pre-trained classifier for detecting frontal faces in images. This classifier is an essential component of our system, as it enables us to detect faces in the video frames captured by the webcam.